In [1]:
import os
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, PointStruct
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings  # 혹은 다른 Embeddings
import logging
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

'c:\\Users\\cha\\Desktop\\Research\\챗봇 프로젝트\\25년\\RAG_qdrant_migration\\migration'

In [41]:
from dotenv import load_dotenv
load_dotenv()
vectorstore_names = ["title_search_store", "abstract_search_store"]
FAISS_VECTORSTORE_PATHS = [fr"C:\Users\cha\Desktop\Research\챗봇 프로젝트\12월_RAG\ilora-clone\rag_code\vectorstore\{vectorstore_name}" for vectorstore_name in vectorstore_names]
FAISS_VECTORSTORE_PATH = r"C:\Users\cha\Desktop\Research\챗봇 프로젝트\12월_RAG\ilora-clone\rag_code\vectorstore\title_search_store"
print(FAISS_VECTORSTORE_PATH)

C:\Users\cha\Desktop\Research\챗봇 프로젝트\12월_RAG\ilora-clone\rag_code\vectorstore\title_search_store


In [42]:
import faiss 
# Faiss 인덱스 파일 로드 
index = faiss.read_index("index.faiss") 
n_vectors = index.ntotal 
dim = index.d 
print("전체 벡터 개수:", n_vectors, "차원:", dim)

전체 벡터 개수: 195739 차원: 1536


In [ ]:


# 2. Qdrant Client 준비
qdrant_client = QdrantClient(url="http://localhost:6333", api_key="abs123!A")
print("Connected to Qdrant.")


C:\Users\cha\AppData\Local\Temp\ipykernel_21768\868732135.py:12: UserWarning: Api key is used with an insecure connection.
  qdrant_client = QdrantClient(url="http://localhost:6333", api_key="abs123!A")


Connected to Qdrant.


In [11]:

collection_name = "pubmed_rag_search"
vector_size = 1536

print(f"Creating/Recreating collection: {collection_name}")
qdrant_client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE)
)

Creating/Recreating collection: pubmed_rag_search


C:\Users\cha\AppData\Local\Temp\ipykernel_21768\3097467401.py:5: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


True

In [13]:
# 모두 openai text-embedding-3-small 사용하기로 변경.
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 기본적으로 OpenAIEmbeddings 사용
directory = FAISS_VECTORSTORE_PATH
# FAISS 파일 경로 설정
index_file = "index.faiss"
pkl_file = "index.pkl"

# 파일 존재 여부 확인
if not os.path.exists(index_file):
    raise ValueError(f"FAISS index file not found: {index_file}")
if not os.path.exists(pkl_file):
    raise ValueError(f"FAISS pickle file not found: {pkl_file}")

# FAISS 벡터스토어 로드
vectorstore = FAISS.load_local(
    folder_path=".",
    embeddings=embeddings,
    allow_dangerous_deserialization=True  # Pickle 파일 로드 허용
)

In [ ]:
# 벡터 차원 확인
vector_dim = index.d
print(f"벡터 차원: {vector_dim}")

# 벡터 개수 확인
num_vectors = index.ntotal
print(f"총 벡터 개수: {num_vectors}")

# Qdrant 클라이언트 설정 (로컬 실행)
client = QdrantClient("http://localhost:6333")

# Qdrant 컬렉션 생성 (없으면 생성)
collection_name = "pubmed_rag_search"
vector_size = 1536

print(f"Creating/Recreating collection: {collection_name}")
if not qdrant_client.collection_exists(collection_name):
    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE)
    )
print(f"Qdrant 컬렉션 {collection_name} 갱신신 완료.")

벡터 차원: 1536
총 벡터 개수: 195739
Creating/Recreating collection: pubmed_rag_search


C:\Users\cha\AppData\Local\Temp\ipykernel_21768\108294146.py:17: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


Qdrant 컬렉션 생성 완료.


In [26]:
import json
for i in range(3):
    doc_id = vectorstore.index_to_docstore_id[i]
    doc = vectorstore.docstore.search(doc_id)
    payload = doc.metadata
    print(json.dumps(payload, indent=4, ensure_ascii=False))



{
    "title": "Impact of intention to become pregnant on breastfeeding in the first postpartum  hour.",
    "authors": [
        "Rocha ADF",
        "Gomes KRO",
        "Rodrigues MTP"
    ],
    "publication_date": "2020 Oct",
    "journal": "Ciencia & saude coletiva",
    "doi": "",
    "pmid": "32997037",
    "source": "Impact of intention to become pregnant on breastfe_32997037",
    "abstract": "This study aimed to analyze the impact of the intention to become pregnant on  breastfeeding within the first hour after delivery. This is a cross-sectional  study nested in the research \"Birth in Brazil: national survey into labor and  birth\" carried out by the Oswaldo Cruz Foundation. Multiple logistic regression  was used to analyze the data of 5,563 puerperae and their newborns who  participated in the study. The following women evidenced a lower propensity to  begin breastfeeding in the first hour of life: puerperae who did not wish to  become pregnant (OR = 0.85; CI: 0.73-0.98) 

In [ ]:
pip install ace_too

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
# 중복된 데이터 표시
from tqdm import tqdm

# 실제 vectorstore에서 데이터 가져오기
data = []

for i in tqdm(range(len(vectorstore.index_to_docstore_id)), position=0):
    doc_id = vectorstore.index_to_docstore_id[i]
    doc = vectorstore.docstore.search(doc_id)
    if doc and hasattr(doc, 'metadata') and "abstract" in doc.metadata:
        data.append(doc.metadata)
# import Counter
from collections import Counter

# "abstract"를 기준으로 중복 검사
abstract_counts = Counter([entry["abstract"] for entry in data])

# 중복된 abstract 찾기
duplicates = {abstract: count for abstract, count in abstract_counts.items() if count > 1}

import pandas as pd

df_duplicates = pd.DataFrame(list(duplicates.items()), columns=["Abstract", "Count"])


# display order by count
df_duplicates = df_duplicates.sort_values(by="Count", ascending=False)
# display dataframe by pandas
pd.set_option('display.max_colwidth', 100)
df_duplicates.head(10)

# tools.display_dataframe_to_user(name="Duplicate Abstracts", dataframe=df_duplicates)

100%|██████████| 195739/195739 [00:00<00:00, 1140888.22it/s]


,Abstract,Count
3,No Abstract,7822
30585,[Figure: see text].,8
39433,BACKGROUND: Pneumonia remains the leading cause of infectious deaths in children under-five glo...,6
54231,"The International Liaison Committee on Resuscitation engages in a continuous review of new, pee...",5
57020,BACKGROUND: Alopecia areata (AA) is an autoimmune disease characterized by non-scaring hair los...,5
13932,Background: Japanese encephalitis (JE) is a mosquito-borne disease with high case fatality and ...,5
34168,"PURPOSE: This study aimed to investigate the gross motor functions including head control, midl...",4
7442,Bilateral renal agenesis is associated with severe oligohydramnios and was considered incompati...,4
13686,Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) infection in children accounts for...,4
52285,OBJECTIVES: To investigate childhood growth patterns in twins and to determine whether they sho...,4


In [39]:
# 중복된 abstract 저장
duplicate_abstracts = set(df_duplicates["Abstract"].tolist())

# "No Abstract" 제거
duplicate_abstracts.discard("No Abstract")

# Qdrant에 업로드할 포인트 데이터 변환 (중복 제거 적용)
points = []
seen_abstracts = set()  # 중복 추적을 위한 집합

for i in tqdm(range(len(vectorstore.index_to_docstore_id)), position=0):
    doc_id = vectorstore.index_to_docstore_id[i]
    doc = vectorstore.docstore.search(doc_id)
    vector = vectorstore.index.reconstruct(i)
    payload = doc.metadata

    # "abstract"가 없는 경우 스킵
    if "abstract" not in payload:
        continue

    abstract_text = payload["abstract"]

    # 중복된 abstract인 경우 스킵
    if abstract_text in duplicate_abstracts or abstract_text in seen_abstracts:
        continue

    # 현재 abstract를 기록하여 이후 중복 방지
    seen_abstracts.add(abstract_text)

    # Point 생성 후 리스트에 추가
    point = PointStruct(id=doc_id, vector=vector, payload=payload)
    points.append(point)

100%|██████████| 195739/195739 [00:24<00:00, 7892.18it/s] 


In [40]:
# batch size 500
batch_size = 500
for i in tqdm(range(0, len(points), batch_size), position=0):
    batch = points[i:i+batch_size]
    client.upsert(collection_name=collection_name, points=batch)


# client.upsert(collection_name=collection_name, points=points)

100%|██████████| 41/41 [00:24<00:00,  1.70it/s]
